In [1]:
!pip install opencv-python

In [ ]:
import cv2
import csv
import json
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
from pathlib import Path

In [ ]:
# w, h
# british:  0           1           2           3           4           5           6           7           8           9           10
#           65, 90      65, 95      65, 100     65, 100     55, 80      65, 110     65, 110     55, 90      50, 85      50, 85      65, 105
#
# raw:      0           1           2           3           4           5           6           7
#           85, 135     95, 165     70, 155     90, 125     125, 225    125, 225    45, 95      115, 195

sizes = {"british": [(65, 90), (65, 95), (65, 100), (65, 100), (55, 80), (65, 110), (65, 110), (55, 90), (50, 85), (50, 85), (65, 105)], 
         "raw": [(85, 135), (95, 165), (70, 155), (90, 125), (125, 225), (125, 225), (45, 95), (115, 195)]}

ratios = {}
with open("ratios.json") as f:
    ratios = json.load(f)

In [ ]:
def get_central_points(mask):
    contours, _ = cv2.findContours( mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE )
    central_points = list()

    for contour in contours:
        try:
            moments = cv2.moments(contour)

            cx = int(moments['m10']/moments['m00'])
            cy = int(moments['m01']/moments['m00'])

            central_points.append([cx, cy])
        except:
            pass

    return central_points

In [ ]:
for key in list(sizes.keys()):
    for num in range(len(sizes[key])):
        for path in tqdm([*Path("clusters/" + key + "/cluster_" + str(num)).glob('*.[J|j][P|p][G|g]')]):
            name = str(path.stem)

            img = cv2.imread("all_cropped_images/" + name + ".jpg")
            mask = cv2.imread("all_cropped_circles/" + name + ".png", cv2.COLOR_BGR2GRAY)

            central_points = get_central_points(mask)

            labels = [0] * len(central_points)
            x_center = []
            y_center = []
            r_width = []
            r_height = []

            for i in range(len(central_points)):
                # x = roi.iloc[i, 0]
                # y = roi.iloc[i, 1]
                x = central_points[i][0]
                y = central_points[i][1]
                
                img = cv2.circle(img, (int(x), int(y)), radius=1, color=(0, 0, 0), thickness=1)
                
                WIDTH = img.shape[1]
                HEIGHT = img.shape[0]
                
                rect_width = sizes[key][num][0] / ratios[name]
                rect_height = sizes[key][num][1] / ratios[name]

                start_point = (int(x - rect_width / 2), int(y - rect_height / 2))
                end_point = (int(x + rect_width / 2), int(y + rect_height / 2))
                
                img = cv2.rectangle(img, start_point, end_point, color=(0, 0, 0), thickness=1)
                
                x_center.append(round(x / WIDTH, 6))
                y_center.append(round(y / HEIGHT, 6))
                r_width.append(round(rect_width / WIDTH, 6))
                r_height.append(round(rect_height / HEIGHT, 6))
                

            # plt.figure(figsize=(10, 10))
            # plt.imshow(img)
            cv2.imwrite("check/" + name + ".png", img)

            data = []
            for k in zip(labels, x_center, y_center, r_width, r_height):
                data.append(k)
            df = pd.DataFrame(data, columns=None)
            df.to_csv("labels/" + name + ".txt", header=None, index=None, sep=' ')